<a href="https://colab.research.google.com/github/alirezamdv/awesome/blob/master/Copy_of_abv_uebung_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# X's and O's detection with a simple CNN (afg-uebung-3)

Please fill in

**Name:** Mahdavi, Alireza

**Email:** alma@uni-bremen.de

**Time needed:** 2 Day

In [0]:
import math
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
# from keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout 
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 16, 25

print (tf.__version__, keras.__version__)

from google.colab import files  # if you use google colab

# The Dataset
The dataset is stored in a .zip file with a .jpg file for each image and corresponding .png file for the ground-truth annotation.

In [0]:
# TODO
gtResolution = "-quarter" # set to "", "-half" or "-quarter" depending on the output resolution of your network

In [0]:
# Download the x-and-o.zip dataset and unzip it
# Alternative: Set data_root if the dataset is already downloaded.
import os
import glob
def loadDataset():
    """Download the dataset and split it in test and training. Chooses the ground-truth
       resoltion depending on gtResolution ='', '-quarter' or ''-half' """
    data_root = keras.utils.get_file('x-and-o.zip','http://www.informatik.uni-bremen.de/agebv2/pub/x-and-o.zip', extract=True)
    data_root = os.path.join(os.path.split(data_root)[0],"x-and-o")
    # print(data_root)
    # data_root = "/root/.keras/datasets/x-and-o" # Use this instead if the dataset is on your system
    dataset = [(os.path.join(data_root,'x-and-o-{:03d}.jpg'.format(i)), os.path.join(data_root,'x-and-o-{:03d}-gt{:}.png'.format(i, gtResolution))) for i in range(0,1000) ]
    dataset = [i for i in dataset if os.path.isfile(i[0]) and os.path.isfile(i[1])]
#     print(dataset)
    #random.shuffle(dataset)
    nTrain = int(math.ceil(0.8*len(dataset)))
    return dataset[0:nTrain], dataset[nTrain:]

def repeatToLength(x, minLen):
   if len(x)<minLen: x = x*math.ceil(minLen/len(x)) 
   return x

train, test = loadDataset()
print(f"Loaded {len(train)+len(test)} images with ground-truth, {len(train)} training, {len(test)} test")

if len(test)==0: test = train.copy() # Hack if we only have one image
train = repeatToLength (train, 100)# Hack, duplicate images, because training progress bar gets confusing otherwise
test  = repeatToLength (test, 4)
print(f"Made {len(train)} training, {len(test)} test from it by duplication.")

In [0]:
def loadImage(fname, color_mode='rgb'):
  "Loads an image as a h*w*3 numpy float array converted to [0..1] as range"
  return img_to_array(load_img(fname,color_mode=color_mode), dtype="float")/255.0

# Load the first image and get the shape of that: All images have the same size
def shapeOfFilename(fname,color_mode='rgb'):
  "Returns the imageshape of fname (filename)."
  imageShape = loadImage(fname, color_mode=color_mode)
  return imageShape.shape

# plt.rcParams['figure.figsize'] = 16, 25
# plt.imshow(loadImage(train[0][0]), aspect='equal', interpolation=None)
# plt.show()

imageShape = shapeOfFilename(train[0][0],color_mode='grayscale')
print(f"Image format {imageShape}.")
gtShape = shapeOfFilename(train[0][1],color_mode='rgb')
print(f"Ground truth format {gtShape}.")
# import warnings
# warnings.filterwarnings('ignore')

In [0]:
# TODO: Implement
def annotation2PropDist (gtImage):
  """The ground truth images in the dataset have red and blue dots on white background.
     This function converts such a gtImage into the format needed for training a classifier
     with the three class (None 0 , X 1 , O 2). The format has three channels, containing the
     probability for background, X and O respectively. The function also works with
    mixed colors as they appear from downsizing the ground-truth image. All images have range [0..1].
    Make sure the result does not exceet the same range."""
  # This function should not use loops but numpy operations, because otherwise training will
  # become excessively slow.
  img = np.array(gtImage)
  h,w,_= img.shape
  shp = h*w  
  BG= img[:,:,1].reshape(shp,1)  #White
  Xs=abs(BG-img[:,:,0].reshape(shp,1)) #Red
  Os=abs(BG-img[:,:,2].reshape(shp,1)) #Blue
  ret = np.hstack((BG,Xs,Os)).reshape(img.shape) # put all togather in orginal shape
  return ret

In [0]:
def showImageWithOverlay (image, labels, filename=None, exaggerate=1):
    """Overlay the labels onto the image. labels may have a lower resolution and must have a three
      channel format showing a probability distribution of (background,X,O). If filename is given,
      the result is saved. The probabilities for X and O are multiplied by exaggerate to allow
      viewing small probabilities too."""
    factor = image.shape[0]//labels.shape[0]
    assert labels.shape[0]*factor==image.shape[0] and labels.shape[1]*factor==image.shape[1]
    assert len(labels.shape)==3 and labels.shape[2]==3
    if len(image.shape)==2: image = image[:,:,np.newaxis] # add third dimension for grayscale
    if image.shape[-1]==1: image = np.repeat(image, 3, axis=-1) # grayscale to color
    labelsResized = np.repeat (labels, factor, axis=0) # y
    labelsResized = np.repeat (labelsResized, factor, axis=1) #x
    displayImage = image * np.maximum(0,(1-exaggerate*(1-labelsResized[:,:,0,np.newaxis])))
    displayImage[:,:,0] += exaggerate*labelsResized[:,:,1] # X-->Red
    displayImage[:,:,2] += exaggerate*labelsResized[:,:,2] # Y-->Blue
    np.clip(displayImage, 0, 1, out=displayImage)
    plt.imshow (displayImage, aspect="equal", interpolation=None)
    if filename is not None:
       plt.savefig(filename, dpi=150, bbox_inches='tight')
    plt.show
    

labels=annotation2PropDist(loadImage(train[0][1]))
# Use the following to inspect individual channels of the labels image for debugging
# 
# plt.imshow(labels)
# plt.imshow (labels[:,:,1])#, vmin=0, vmax=1, aspect="equal", interpolation=None, cmap="gray")
# plt.show
plt.rcParams['figure.figsize'] = 16, 25
showImageWithOverlay (loadImage(train[0][0],color_mode='grayscale'), labels)
# labels[19][126]
# plt.imshow(loadImage(train[0][1]))
# plt.show()
# np.average(labels, axis=1)
# np.average(labels,axis=1)


In [0]:
# TODO: implement
def entropy(x):
    '''Returns the average entropy of the probability distributions in x. The last axis of x
    is assumed to represent the different events with their probabilities. Over this axis the
    entropy is computed, all other axes create just a multitude of such distributions and over
    these axes the average is taken.'''
    # -(np.nansum(x*np.log(x)))/len(x)
    return np.max(np.nan_to_num(-tf.math.cumsum(x*tf.math.log(x))))  

In [0]:
# TODO: Implementierung
      # [1,0,0]
s = np.array([[1,0,0,0], [0.5,0.5,0,0]])
entropy(s)

In [0]:
# Run this test to check if entropy is coded correctly
np.testing.assert_almost_equal (entropy(np.array([[1,0,0,0], [0.5,0.5,0,0]],dtype=np.float)), math.log(2)/2)

In [0]:
print(f"A perfect classifier could reach a loss of {entropy(labels)} on this dataset")
print(f"A classifier not looking at the data at all could reach {entropy(np.average(labels,axis=(0,1)))} on this dataset") 
#TODO: Comment, what loss do you expect to achieve?
#0.009

In [0]:
# Tools to load the training / test set on the fly, so the whole dataset
# doesn't need to be kept in memory.
def prepareEntry (entry):
  """Dummy function to prepare and entry of the dataset. It takes one entry
     and converts it to a input, ground-truth output pair that is given
     to keras. At the moment the image is loaded and the output is just empty."""
  #print(f"Loading....{entry[0]}")
  return (loadImage(entry[0], color_mode='grayscale').astype('float32'), annotation2PropDist(loadImage(entry[1])).astype('float32'))

class DataGenerator(keras.utils.Sequence):
     """Provides a dataset to keras in a load on demand fashion. The dataset is a list
     of entries, each of which is finally converted via prepareEntry
     Adapted from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly"""
    
     def __init__(self, dataset, prepareEntry=prepareEntry, batch_size=4, shuffle=True):  
        'Initialization'
        self.dataset = dataset
        self.prepareEntry = prepareEntry
        self.batch_size = batch_size
        self.shuffle = shuffle
        
     def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.dataset) / self.batch_size))
      
     def __getitem__(self, index):
          'Generate one batch of data'
          startIdx = index*self.batch_size
          endIdx = min(startIdx + self.batch_size, len(self.dataset))
          batchFilenames = self.dataset[startIdx:endIdx]
          batchData = [self.prepareEntry(e) for e in batchFilenames]
          for e,n in zip(batchData, batchFilenames):
            if e[0].dtype!='float32' or e[0].shape!=imageShape:
                raise Exception(f'{n[0]} has datatype {e[0].dtype} and format {e[0].shape}')
            if np.min(e[0])<0 or np.max(e[0])>1:
                raise Exception(f'{n[0]} has range [{np.min(e[0])}..{np.max(e[0])}]')
            if e[1].dtype!='float32' or e[1].shape!=gtShape:
                raise Exception(f'{n[1]} has datatype {e[1].dtype} and format {e[1].shape}')
            if np.min(e[1])<0 or np.max(e[1])>1:
                raise Exception(f'{n[1]} has range [{np.min(e[1])}..{np.max(e[1])}]')
          X = np.array([e[0] for e in batchData])
          y = np.array([e[1] for e in batchData])
          return X, y
        
     def on_epoch_end(self,logs={}):
         'Updates indexes after each epoch'
         if self.shuffle: random.shuffle(self.dataset)
     def on_batch_end(self,logs={}):
         'Updates indexes after each epoch'
         if self.shuffle: random.shuffle(self.dataset)
         THR = 0.0013 #Assign THR with the value at which you want to stop training.
         currentLoss = logs.get('loss')
         if currentLoss != None:
            if currentLoss <= THR:
                 self.stop_training = True

In [0]:
# Use this to test that the data generator works and in particular that the format,
# datatypes and value ranges are correct
def showEntryOfGenerator (dataGen, index=0):
  """Fetches the first batch, prints dataformat statistics and 
  shows the first entry both as image X and annotation y."""
  X, y = dataGen[index]
  print(f"X has shape{X.shape}, type {X.dtype} and range [{np.min(X)}..{np.max(X)}]")
  print(f"y has shape{y.shape}, type {y.dtype} and range [{np.min(y)}..{np.max(y)}]")
  #for i in range(0,X.shape[0]):
  #   showImageWithOverlay (X[i], y[i]) 
  #   plt.show()

trainGen = DataGenerator (train)
testGen = DataGenerator (test)
for i in range(0,len(trainGen)):
    
    showEntryOfGenerator (trainGen, index=i)
for i in range(0,len(testGen)):
    showEntryOfGenerator (testGen, index=i)


In [0]:
Now construct a reasonable network (model) to solve the problem. You can start with the architecture from
Aufgabe 2.3 (3*3-Pool-3*3-Pool-3*3), however my experiments showed that 5 channels are too few and 7 needed. Take care in the final
layer to choose the right activation. This also may have consequences on the preceding layer als well. Also
take care to choose the right loss and metric. In my experiments I needed about 100 epochs to get a reasonable result (Training Loss 0.015). This took several hours on my 3 GHz Intel Core i7 computer without GPU.
Using GPU or google colab is much faster. The network was unable to detect larger Os. You might also try a somewhat larger network (more layers, more channels, maybe larger filter in the first layer).

Take a look at https://keras.io/api/ for documentation and code examples.

Use model.summary() to print the model

In [0]:
# TODO: Implementierung
lr=0.001
model =Sequential()

model.add(Conv2D( 7 , kernel_size=(3,3),data_format='channels_last'  ,activation='relu',strides=(1,1),padding='same',input_shape=(1160,820,1) ))

model.add(MaxPooling2D(pool_size=(2, 2), padding='same',input_shape=(1160,820,7)))

model.add(Conv2D( 7, kernel_size=(3,3),data_format='channels_last' ,activation='relu',strides=(1,1),padding='same',input_shape=(580,410,7)))
 
model.add(MaxPooling2D(pool_size=(2, 2), padding='same',input_shape=(580,410,7)))

model.add(Conv2D( 7, kernel_size=(3,3),data_format='channels_last' ,activation='relu',strides=(1,1),padding='same',input_shape=(290,205,7)))

model.add(Dense(units = 3, activation = 'softmax'))


# model.add(Flatten())
# model.add(Dropout(0.5))

# model.add(Dense(units = 128, activation = 'sigmoid'))
# model.add(Dense(units = 3, activation = 'softmax'))
# optimizer=keras.optimizers.Adam(learning_rate=0.001)

model.build()
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])



In [0]:
# Train the Network
#model.load_weights ("x-and-o-net.h5"), #load a previous checkpoint
for ctr in range(40):
  model.fit_generator(generator=trainGen, epochs=20, validation_data=testGen)
  model.save_weights("x-and-o-net.h5") # saves weights (e.g. a checkpoint) locally
  print(f'###### Nr.{ctr} ####### ')

# files.download('x-and.o.net.h5') # download weights from e.g. google-colab to local machine

In [0]:
model.save_weights("x-and-o-net.h5")
files.download('x-and-o-net.h5') # download weights from e.g. google-colab to local machine

In [0]:
# Use this to visualize results
# It shows the input image (background) and predicted heatmap (colorful labels)
testIdx = random.randint(0,len(testGen)-1)
testBatch = testGen[testIdx]
plt.rcParams['figure.figsize'] = 16, 25
print (testIdx, testBatch[0].shape)
yHats = model.predict (testBatch[0])
# yHats=tf.argmax(yHats)
print(model.evaluate(testBatch[0], testBatch[1], batch_size=4))
s = tf.argmax(yHats[0:,:,:])
for i in range(0,len(testBatch[0])):
   showImageWithOverlay (testBatch[0][i], yHats[i], filename=f"x-and-o-result-{i}.png", exaggerate=2) 
   plt.show()
# entropy(yHats[0])
# plt.imshow(s)
# plt.show()
# s[50: ,50:,0]
# testBatch[0][1].shape

In [0]:
channel = 0
np.max(yHats[0,:,:,channel]) # Use this to inspect maximum probability for Nothing (channel 0), X (1) or O (2)

Give a comment on the performance of your network (can be German).\
'zuerst versuchte ich, die annotation2propdist mit numpy **mask** zu implementieren(war gar nicht gut mit dem quarter Bild) und dann mit numpy **apply_along_axis**, aber es war nicht schnell genug... mit der aktuellen Implementierung, nämlich Numpy **slicing**, konnte ich es für 800 Epochen laufen lassen und ich denke, die Performance ist wirklich sehr gut.'